In [1]:
import matplotlib
import rebound
import numpy as np
import time

# Initialize the simulation
sim = rebound.Simulation()
sim.integrator = "whfast" #IMPORTANT: Chose WHFast for quicker simulation, but IAS15 may be more accurate
sim.dt=1e-3 #Must be small enough for accuracy, but large enough so simulation isn't slow
sim.widget(port=1234, size=(800,800)) #Visualization
sim.softening = 1e-6 #softening parameter prevents singularities from occuring when particles get too close, may remove

date = "2025-01-22 18:00"
# Add Sun
sim.add("Sun", date=date)

# Add Jupiter
sim.add("Jupiter",date=date)

sim.move_to_com()

No=10000

# Precompute Jupiter's position
jupiter = sim.particles[1]
jupiter_x, jupiter_y, jupiter_z = jupiter.x, jupiter.y, jupiter.z

# Generate test particle positions and velocities, WE ASSUME RANDOM DISTRIBUTION, up to 1 AU away from Jupiter
r0 = np.random.random(No)+1.76e-5  # Random radial distances in [r_min,1+r_min)
theta0 = np.random.random(No) * np.pi  # Random polar angles
phi0 = np.random.random(No) * 2 * np.pi  # Random azimuthal angles

# Convert to Cartesian coordinates (relative to Jupiter)
x0 = r0 * np.cos(phi0) * np.sin(theta0)
y0 = r0 * np.sin(phi0) * np.sin(theta0)
z0 = r0 * np.cos(theta0)

x1 = jupiter_x + x0
y1 = jupiter_y + y0
z1 = jupiter_z + z0

# Define the velocity range in G=1 units, based off of astronomer estimates, MAY CHANGE
v_min = 0.0671  # Minimum velocity
v_max = 3.357   # Maximum velocity

# Generate velocities
q = np.random.random(No) * 2 * np.pi  # Direction parameter, MAY WANT TO TWEAK DISTRUBUTION, assume random here
v = np.random.uniform(v_min, v_max, No)  # Velocity magnitudes, MAY WANT TO TWEAK DISTRIBUTION, we assume uniform here

# Calculate velocity components
with np.errstate(divide='ignore', invalid='ignore'):  # Handle division by zero
    magb1 = np.sqrt(x0**2 + y0**2)
    magb2 = np.sqrt((x0**2) * (z0**2) + (y0**2) * (z0**2) + ((x0**2) + (y0**2))**2)

    # Default velocity components
    v0x = (-v * y0 * np.cos(q)) / magb1 + (-v * x0 * z0 * np.sin(q)) / magb2
    v0y = (v * x0 * np.cos(q)) / magb1 + (-v * y0 * z0 * np.sin(q)) / magb2
    v0z = v * np.sin(q) * ((x0**2) + (y0**2))**2 / magb2

    # Special case: z0 == 0
    special = z0 == 0
    magb1[special] = x0[special]
    magb2[special] = np.sqrt((x0[special]**2) * (y0[special]**2) + (x0[special]**4))

    v0x[special] = -v[special] * x0[special] * y0[special] * np.sin(q[special]) / magb2[special]
    v0y[special] = v[special] * (x0[special]**2) * np.sin(q[special]) / magb2[special]
    v0z[special] = -v[special] * x0[special] * np.cos(q[special]) / magb1[special]

#velocity is such that it is a random vector orthogonal to the radial seperation between the comet and Jupiter, such that t=0 is closest approach

T=10

boundp = np.zeros(No, dtype=bool)
boundn = np.zeros(No, dtype=bool)

# Add all test particles
for i in range(No):
    sim.add(m=1e-21,x=x1[i], y=y1[i], z=z1[i], vx=v0x[i], vy=v0y[i], vz=v0z[i])
    sim.integrate(T)
    boundp[i]=sim.particles[2].e<=1
    sim.integrate(-T)
    boundn[i]=sim.particles[2].e<=1
    sim.remove(2)
    
counter = 0
for i in range(No):
    if boundp[i] and not boundn[i]:
        counter+=1
        
bound_fraction = counter / No
rate_of_entry=10000 #interstellar objects entering solar system (Neptune inner) per year, may want to refine?
probability=2/2700 #rough estimate of likelihood of encounter with Jupiter, calculated by volume of ball we conisder around
#Jupiter divided by cylindrical volume of solar system up to Jupiter with height 2 AU, MORE RESEARCH NEEDED
print(f"Bound fraction: {bound_fraction}, Capture Rate (per year) Estimate: {rate_of_entry*probability*bound_fraction}")

Searching NASA Horizons for 'Sun'... 
Found: Sun (10) 
Searching NASA Horizons for 'Jupiter'... 
Found: Jupiter Barycenter (5) (chosen from query 'Jupiter')
Bound fraction: 0.0001, Capture Rate (per year) Estimate: 0.0007407407407407408
